### -------------------------------------------RUN THIS BLOCK ONLY ONCE------------------------------------------------------
#### TO GET "jpg_path", "pt2d", "pose_para"

In [2]:
from os import walk
DATASET_PATH = '/home/ubamba98/PROJECTS/dataset/300W_LP/Dataset/'#Path to dataset for training

def Loading_filepath(PATH):
    '''
    File directory generator.
    
    ---INPUT---
    
    PATH --- Directory to the dataset
    
    ---OUTPUT---
    
    jpgs --- list of path to images
    
    mat  --- list of path to corresponding mat files
    
    ** Made for loading 300W_LP dataset **
    
    '''
    jpgs = []
    mat = []
    for (dirpath,dirnames,filenames) in walk(PATH):
        for filename in filenames:
            if(filename[-3:]=='jpg'):
                jpgs.append(dirpath+"/"+filename)
                mat.append(dirpath+"/"+filename[:-3]+"mat")
    return jpgs,mat
jpg_path,mat_path = Loading_filepath(DATASET_PATH)
print("Dataset size: ",len(jpg_path))

Dataset size:  122450


In [3]:
from scipy.io import loadmat
from IPython.display import clear_output
def extract_mat(mat_file_path):
    '''
    Returns 2D_landmarks,Pose_para
    
    ---INPUT---
    
    mat_file_path --- Path to mat file
    
    ---OUTPUT---
    
    pt2d --- 2D_landmarks for corresponding image
    
    Pose_Para --- Pose parameters for corresponding image
    
    **Pose_Para unfoldes to [pitch, yaw, roll, tdx, tdy, tdz, scale_factor]
    
    ** Made for loading 300W_LP dataset **
    
    '''
    matfile=loadmat(mat_file_path)
    pt2d = matfile['pt2d']
    Pose_Para = matfile['Pose_Para'][0][:3]
    return pt2d,Pose_Para
pt2d = []
pose_para = []
j=0
for i in mat_path:
    j+=1
    p,po = extract_mat(i)
    po = po[0:3]
    pt2d.append(p)
    pose_para.append(po)
    clear_output()
    print("Done: ",j/122450 *100)

Done:  100.0


In [4]:
import numpy as np
np.save("jpg_path.npy",jpg_path)
np.save("pt2d.npy",pt2d)
np.save("pose_para.npy",pose_para)

### ---------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import numpy as np

In [2]:
jpg_path = np.load("./jpg_path.npy")
pt2d = np.load("./pt2d.npy")
pose_para = np.load("./pose_para.npy")

In [3]:
train_jpg_path   = jpg_path[:110205]
train_pt2d   = pt2d[:110205]
train_pose_para   = pose_para[:110205]

In [4]:
test_jpg_path   = jpg_path[110205:]
test_pt2d   = pt2d[110205:]
test_pose_para   = pose_para[110205:]

In [5]:
from keras.applications.mobilenetv2 import MobileNetV2
from keras.layers import Dense,Flatten
from keras.models import Model,Sequential
from keras.utils import Sequence
shape = (160,160,3)
base_model = MobileNetV2(input_shape=shape,include_top=False)
x = base_model.output
x = Flatten()(x)
x = Dense(100,activation='relu')(x)
x = Dense(50,activation='relu')(x)
x = Dense(100,activation='relu')(x)
predictions = Dense(3,activation=None)(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

/home/ubamba98/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [6]:
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [7]:
import cv2
class Generator(Sequence):
    def __init__(self,img_path, pose_para, pt2d, batch_size, shape = (160,160)):
        self.img_path = img_path
        self.pose_para = pose_para
        self.pt2d = pt2d
        self.batch_size = batch_size
        self.shape = shape
    def crop_image(self,path, p2d):
        img = cv2.imread(path)
        x_min = int(min(p2d[0,:]))-20
        y_min = int(min(p2d[1,:]))-20
        x_max = int(max(p2d[0,:]))+15
        y_max=  int(max(p2d[1,:]))+20
        crop_image = img[y_min:y_max, x_min:x_max]
        crop_image = cv2.resize(crop_image,self.shape)
        return crop_image/225.0
    def __len__(self):
        return int(np.ceil(len(self.img_path)/float(self.batch_size)))
    def __getitem__(self,idx):
        batch_x = self.img_path[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.pose_para[idx*self.batch_size:(idx+1)*self.batch_size]
        pt2d_ = self.pt2d[idx*self.batch_size:(idx+1)*self.batch_size]
        return np.array([self.crop_image(path,p2d) for (path,p2d) in zip(batch_x,pt2d_)]), np.array(batch_y)

In [8]:
train_gen = Generator(img_path=train_jpg_path, pose_para=train_pose_para, pt2d=train_pt2d, batch_size=32,shape=(160,160))
test_gen = Generator(img_path=test_jpg_path, pose_para=test_pose_para, pt2d=test_pt2d, batch_size=32,shape=(160,160))

model.fit_generator(generator=train_gen,steps_per_epoch=(110205//32),epochs=2,validation_data=test_gen,validation_steps=(12245//32),use_multiprocessing=True,max_queue_size=32)

Epoch 1/2
3443/3443 [==============================] - 3417s 993ms/step - loss: 0.0228 - acc: 0.9201 - val_loss: 0.0324 - val_acc: 0.9170
Epoch 2/2
3443/3443 [==============================] - 3339s 970ms/step - loss: 0.0103 - acc: 0.9400 - val_loss: 0.0127 - val_acc: 0.9374


In [9]:
model.save_weights('weights.h5')

In [10]:
model.save('model.h5')